In [2]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git@64f63a7df2a02cfd144592d9aa9c871b59258c55
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 30.9 MB/s eta 0:00:0000:0100:01


In [1]:
from peft import PeftModel
from transformers import AutoTokenizer, BloomForCausalLM
import torch

BASE_MODEL = "bigscience/bloomz-7b1-mt"
LORA_WEIGHTS = "bloomz-gpt"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = BloomForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map = {'': 0},
)
model = PeftModel.from_pretrained(model, LORA_WEIGHTS, device_map={'': 0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


In [2]:
def make_prompt(instruction):
        return f"""Hãy viết một phản hồi thích hợp cho chỉ dẫn dưới đây.

### Instruction:
{instruction}

### Response:"""

In [3]:
def evaluate(q):
    input_ids = tokenizer(make_prompt(q), return_tensors="pt")["input_ids"].to('cuda')
    with torch.no_grad():
        gen_tokens = model.generate(
            input_ids=input_ids,
            max_new_tokens = 256,
            do_sample=True,
            temperature=0.5,
            top_k=20,
            repetition_penalty=1.2,
            #eos_token_id=0, # for open-end generation.
            #pad_token_id=tokenizer.eos_token_id,
        )
    origin_output = tokenizer.batch_decode(gen_tokens)[0]
    output = origin_output.split("### Response:")[1].strip()
    return output

In [ ]:
# from transformers import GenerationConfig
# generation_config = GenerationConfig(
#     temperature=0.2,
#     top_p=0.75,
#     num_beams=4,
# )

# def evaluate(instruction):
#   prompt = make_prompt(instruction)
#   inputs = tokenizer(prompt, return_tensors="pt")
#   input_ids = inputs["input_ids"].cuda()
#   generation_output = model.generate(
#       input_ids=input_ids,
#       generation_config=generation_config,
#       return_dict_in_generate=True,
#       output_scores=True,
#       max_new_tokens=256
#   )
#   for s in generation_output.sequences:
#       output = tokenizer.decode(s)
#       print(output)
#       result = output.split("### Response:")[1].strip()
#       return result

In [4]:
evaluate("Nói cho tôi biết về alpaca")

'Alpacas là những con vật được nuôi để lấy lông, có nguồn gốc từ dãy núi Andes ở Nam Mỹ. Chúng thường cao khoảng 2 feet và nặng hơn 50 pound mỗi lần. Các nhà khoa học đã phát hiện ra rằng loại động vật này sinh sản rất nhanh chóng, với nhiều cặp giao phối trong năm đầu tiên của chúng.</s>'

In [5]:
evaluate("Kể cho tôi nghe về tổng thống Mexico năm 2019")

'Tổng thống của Mexico vào tháng 9 năm 2020 là Andrés Manuel López Obrador. Ông đã được bầu cử trong nhiệm kỳ thứ hai và trở thành người đầu tiên nắm giữ chức vụ này kể từ khi chính phủ quân sự cuối cùng kết thúc sau cuộc cách mạng giành độc lập khỏi thực dân Tây Ban Nha (1821). Lopez Obrador, đảng viên Đảng Cách mạng Công nhân, cũng từng làm thị trưởng Thành phố Mexico trước đó.\nThành tựu nổi bật nhất mà ông theo đuổi trong suốt thời gian ở Nhà Trắng bao gồm việc loại bỏ các công ty dầu khí lớn như Repsol YPF và Sinopec ra khỏi đất nước, đồng ý với thỏa thuận hạt nhân lịch sử với Hoa Kỳ để đảm bảo điện năng an toàn hơn cho quốc gia, đưa vấn đề biến đổi khí hậu lên ưu tiên hàng đầu của mình, tăng lương'

In [6]:
evaluate("Hãy sáng tác 1 bài thơ lục bát")

'Một ngày mưa, lá cây xào xạc,\nĐất ẩm ướt và tôi đứng bên ngoài.\nBầu trời xám xịt không có gì để nói về nó;\nTôi cảm thấy như mình đang bị mắc kẹt trong bóng tối.</s>'

In [7]:
evaluate("Hãy sáng tác giúp tôi 1 bài nhạc rap")

'Tôi có thể cung cấp các lời khuyên về cách tạo ra một bài hát rap thành công. Đầu tiên, hãy chắc chắn rằng bạn hiểu rõ chủ đề của mình và làm nổi bật nó trong giai điệu của bài hát bằng cách sử dụng sự kết hợp giữa âm thanh và hình ảnh để truyền tải thông điệp phù hợp với khán giả mục tiêu của bạn. Thứ hai, đừng quên nhấn mạnh vào phong thái của giọng nói - điều này sẽ khiến người nghe quan tâm hơn đến câu chuyện mà bạn đang muốn kể! Cuối cùng, hãy thử nghiệm nhiều bản phối khác nhau trên mẫu hoặc bảng nhịp trước khi đưa mọi thứ lên sân khấu và tận hưởng niềm vui của việc phát triển sản phẩm cuối cùng của riêng bạn!</s>'

In [8]:
evaluate("ăn gì cho đẹp da")

'Một chế độ ăn uống lành mạnh có thể giúp bạn trông và cảm thấy tốt hơn. Nó bao gồm trái cây, rau quả, protein nạc (như gà hoặc cá), ngũ cốc nguyên hạt và các loại thực phẩm chứa chất chống oxy hóa khác nhau như cà rốt và việt quất. Điều quan trọng là phải tránh đồ ngọt và quá nhiều chất béo bão hòa cũng được tìm thấy trong thịt đỏ và chiên.</s>'

In [9]:
evaluate("ngày của mẹ là ngày mấy?")

'Ngày của Mẹ được tổ chức vào thứ bảy đầu tiên sau Ngày sinh nhật, theo lịch năm 2021.</s>'

In [10]:
evaluate("cách nấu món rau câu?")

'Để làm đồ ăn nhẹ, hãy trộn kem chua với sữa và đường. Đổ hỗn hợp vào khuôn bánh quy hoặc cốc thủy tinh và để đông lạnh trong ít nhất 3-4 giờ trước khi lấy ra khỏi tủ đá bằng dao kéo cứng. Để nó phục vụ trên khăn giấy ẩm được lót khay nướng.</s>'

In [11]:
evaluate("Nếu bạn có thể giúp tôi viết một email mời bạn bè của tôi đi ăn tối vào thứ Sáu, tôi sẽ rất cảm kích.")

'Xin chào [Tên]! Tôi là [Tên] và tôi đang cố gắng tìm ra cách tốt nhất để gửi tin nhắn điện tử đến các liên hệ cá nhân. Có điều gì khác mà chúng ta có thể làm trong thời gian này không?</s>'

In [12]:
evaluate("Viết một đoạn mô tả ngắn về bộ phim: The Witcher (2019)")

'Bộ phim được thực hiện bởi nhà làm phim người Ba Lan, Andrzej Sapkowski và dựa trên cuốn sách cùng tên của anh ấy. Bộ phim kể câu chuyện của Geralt of Rivia - Một hiệp sĩ ma thuật mạnh mẽ sống ở xứ Wales với ước mơ trở thành thợ săn huyền thoại của mình khi giết chết những con rồng khổng lồ tàn nhẫn. Trong quá trình tìm kiếm sự thật ẩn giấu trong cuộc đời ông, cô đã gặp phải nhiều kẻ thù khác nhau như Grimbold, Orc và cả Wiglaf, thủ lĩnh băng giá lạnh lùng nhất mà nhân loại từng biết đến. Phim là một pha trộn độc đáo giữa các diễn viên lồng tiếng chuyên nghiệp và tài tử truyền cảm hứng từ nền văn học châu Âu cổ điển.</s>'

In [14]:
evaluate("Dịch sang tiếng Việt: Je t’aime.")

'Tôi yêu bạn.</s>'

In [17]:
evaluate("Suggest at least five related search terms to \"Mạng neural nhân tạo\"")

'1. Mạng thần kinh - mô hình mạng, học máy và các ứng dụng khác.\n2 . Nền tảng nghiên cứu trí tuệ nhân tạo (AI) như Google DeepMind AI Platform hoặc TensorFlow\n3. Mã hóa神经 khoa học - kiến \u200b\u200bthức về mã hóa dữ liệu và cách nó liên quan đến việc xây dựng hệ thống AI.\n4. Trí tuệ nhân tạo trong lĩnh vực y tế - Hệ thống hỗ trợ chẩn đoán bệnh tự động dựa trên cơ sở dữ liệu lớn có thể được sử dụng để phát hiện sớm nhiều loại ung thư.\n5. Machine Learning với Python - Một ngôn ngữ lập trình phổ biến cho người mới bắt đầu và chuyên gia ML</s>'

In [19]:
evaluate("What is the meaning of life? Explain it in 5 paragraphs.")

'Life means to live an active, productive, and fulfilling lifestyle. Life can be defined as a series of experiences that we have throughout our lifetime with others and ourselves. This includes things like family relationships, friendships, work, education, spirituality, leisure time, hobbies, health care, physical activity, personal growth, và more. The purpose of life is ultimately about being happy with what you are doing on your own terms.</s>'

In [18]:
evaluate("Write a fairy tale about a troll saving a princess from a dangerous dragon. The fairy tale is a masterpiece that has achieved praise worldwide and its moral is \"Heroes Come in All Shapes and Sizes\". Story (in Spanish):")

'Một ngày nọ, ở nơi nào đó trên thế giới có một cô gái xinh đẹp tên là Belle đã bị bắt cóc bởi rồng hung dữ và bị giam giữ trong lâu đài của mình để cứu nó khỏi cái chết.\nKhi thấy tình hình trở nên tồi tệ hơn khi không thể tìm ra con đường thoát thân khác ngoài việc hy sinh bản thân anh ta vì người mà anh ấy yêu thương nhất - nàng tiên nữ nhỏ bé này, ông quyết định mạo hiểm vào những nguy hiểm lớn lao để bảo vệ người phụ nữ đáng yêu của mình với sự giúp đỡ từ nhiều bạn bè cũ của mình như Trolls'

In [4]:
#@title GUI
import gradio as gr
examples = [
    ["Dịch sang tiếng việt: Je t’aime."],
    ["Suggest at least five related search terms to \"Mạng neural nhân tạo\"."],
    ["Write a fairy tale about a troll saving a princess from a dangerous dragon. The fairy tale is a masterpiece that has achieved praise worldwide and its moral is \"Heroes Come in All Shapes and Sizes\". Story (in Vietnamese):"],
    ["Nói cho tôi biết về alpaca"],
    ["Viết một đoạn mô tả ngắn về bộ phim đã cho: The Witcher (2019)"],
    ["Write a Python program that prints the first 10 Fibonacci numbers."],
    ["Ăn gì cho đẹp da"],
    ["Sản phẩm này tốt lắm. Bạn sẽ đánh giá câu trên là tích cực, trung lập hay tiêu cực?"],
    ["Hãy sáng tác 1 bài thơ lục bát về biển cả"],
    ["ngày của mẹ là ngày mấy?"],
    ["cách nấu món rau câu"],
    ["hãy lên kế hoạch đi Đà Lạt trong 3 ngày cho tôi"],
    ["liệt kê mẫu son môi hot nhất hiện nay"],
    ["liệt kê những bài hát của Ưng Hoàng Phúc"],
    ["Viết một email mời bạn bè của tôi đi ăn tối vào thứ Sáu"],
    ["\"Read csv file using pandas and display first ten rows\" Please write code following the instructions in jupyter notebook style."],
    ["ngày nhà giáo việt nam là ngày bao nhiêu"],
    ["Ý nghĩa của cuộc sống là gì?"],
    ["Q: Can Obama have a conversation with George Washington? Give the rationale before answering"],
    ["Q: Obama có thể nói chuyện với George Washington không? Đưa ra lý do trước khi trả lời"],
    ["Đề xuất một số hoạt động hữu ích hoặc cách bắt đầu cuộc trò chuyện để sử dụng tại các buổi gặp mặt nhằm phá vỡ khoảng cách."],
]

title = "Bloomz-GPT"


def inference(instruction):
  return evaluate(instruction)

io = gr.Interface(
  inference,
  inputs=[gr.Textbox(lines=3)],
  outputs=[
    gr.Textbox(lines=3, label="GPT-J Demo")
  ],
  title=title,
  examples=examples
)
io.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://3cd5bf307a51096da2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
